In [5]:
import pyupbit
import time
import json
import requests
import pandas as pd
import mysql.connector
from datetime import datetime 

##### DB연결

In [30]:
# MySQL 서버에 연결
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="1234",
    database="deepCoin"
)

# 연결이 제대로 되었는지 확인
if mydb.is_connected():
    print("MySQL 데이터베이스에 성공적으로 연결되었습니다.")
else:
    print("MySQL 데이터베이스 연결에 실패했습니다.")

# 커서 생성
mycursor = mydb.cursor()


MySQL 데이터베이스에 성공적으로 연결되었습니다.


##### DB 종목마스터 가져오기

In [7]:
# SELECT 쿼리 실행
mycursor.execute("SELECT * FROM c_krw_mst")

# 결과 가져오기
result = mycursor.fetchall()

# 튜플의 0번째 인덱스만 가져와서 마켓 리스트 생성
market_list = [t[0] for t in result]

##### [과거데이터] 최초 종목별 일봉데이터 가져오기

In [31]:
# KRW 종목 리스트 가져오기
columns = ['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']

# 삽입할 테이블 이름
table_name = "c_krw_day"
for market in market_list:
    df = pyupbit.get_ohlcv_from(ticker=market, interval="day", fromDatetime=None,
                   to=None, period=0.5)
    df['date'] = df.index.strftime('%Y-%m-%d')
    df['market'] = market
    df = df[['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']]

    tuple_list = [tuple(x) for x in df.to_numpy()]

    # 데이터 삽입 쿼리 생성
    sql = "INSERT INTO {} ({}) VALUES ({})".format(
        table_name,
        ', '.join(columns),
        ', '.join(['%s'] * len(columns))
    )
    # 데이터 삽입
    mycursor.executemany(sql, tuple_list)
    print(mycursor.rowcount, f"{market} 레코드가 삽입되었습니다.")

# 변경 사항 커밋
mydb.commit()
print("전체 레코드가 삽입되었습니다.")

840 KRW-1INCH 레코드가 삽입되었습니다.
840 KRW-AAVE 레코드가 삽입되었습니다.
2319 KRW-ADA 레코드가 삽입되었습니다.
1755 KRW-AERGO 레코드가 삽입되었습니다.
1297 KRW-AHT 레코드가 삽입되었습니다.
818 KRW-ALGO 레코드가 삽입되었습니다.
1780 KRW-ANKR 레코드가 삽입되었습니다.
483 KRW-APT 레코드가 삽입되었습니다.
1184 KRW-AQT 레코드가 삽입되었습니다.
321 KRW-ARB 레코드가 삽입되었습니다.
2313 KRW-ARDR 레코드가 삽입되었습니다.
2330 KRW-ARK 레코드가 삽입되었습니다.
58 KRW-ASTR 레코드가 삽입되었습니다.
1748 KRW-ATOM 레코드가 삽입되었습니다.
736 KRW-AVAX 레코드가 삽입되었습니다.
1045 KRW-AXS 레코드가 삽입되었습니다.
2032 KRW-BAT 레코드가 삽입되었습니다.
2330 KRW-BCH 레코드가 삽입되었습니다.
232 KRW-BLUR 레코드가 삽입되었습니다.
1288 KRW-BORA 레코드가 삽입되었습니다.
1916 KRW-BSV 레코드가 삽입되었습니다.
2333 KRW-BTC 레코드가 삽입되었습니다.
2277 KRW-BTG 레코드가 삽입되었습니다.
1840 KRW-BTT 레코드가 삽입되었습니다.
1147 KRW-CBK 레코드가 삽입되었습니다.
699 KRW-CELO 레코드가 삽입되었습니다.
1398 KRW-CHZ 레코드가 삽입되었습니다.
1734 KRW-CRE 레코드가 삽입되었습니다.
1272 KRW-CRO 레코드가 삽입되었습니다.
64 KRW-CTC 레코드가 삽입되었습니다.
2022 KRW-CVC 레코드가 삽입되었습니다.
1310 KRW-DKA 레코드가 삽입되었습니다.
1085 KRW-DOGE 레코드가 삽입되었습니다.
1227 KRW-DOT 레코드가 삽입되었습니다.
293 KRW-EGLD 레코드가 삽입되었습니다.
1944 KRW-ELF 레코드가 삽입되었습니다.
2155 KRW-EOS 레코드가 삽입되었습니다

In [22]:
df = pyupbit.get_ohlcv_from(ticker='KRW-ADA', interval="day", fromDatetime=None,
                   to=None, period=0.5)
values = df.values.tolist()

In [23]:
values

[[23.8, 24.0, 20.8, 21.3, 0.0333, 0.74874],
 [21.2, 24.2, 19.9, 23.7, 0.052, 1.15705],
 [23.7, 26.8, 22.7, 23.2, 0.052, 1.26314],
 [23.1, 26.1, 23.1, 25.3, 0.0519, 1.29628],
 [25.3, 25.7, 24.4, 24.7, 0.0514, 1.29012],
 [24.7, 26.1, 24.4, 26.0, 0.0513, 1.2915],
 [26.0, 31.2, 25.0, 30.9, 0.0512, 1.3958],
 [30.7, 39.2, 29.2, 38.2, 0.0512, 1.66595],
 [38.4, 43.2, 33.2, 36.7, 0.0515, 1.96025],
 [36.8, 38.2, 32.5, 33.9, 0.0515, 1.81292],
 [33.6, 34.7, 32.1, 32.5, 0.0503, 1.66899],
 [32.5, 33.3, 28.2, 31.3, 0.0593, 1.82535],
 [31.1, 31.8, 28.7, 30.5, 76.0156, 2271.27987],
 [30.7, 31.3, 29.6, 31.1, 233.0, 7031.1],
 [31.0, 34.5, 30.4, 33.2, 64.86, 2002.562],
 [31.3, 33.8, 31.0, 31.8, 2.11, 68.407],
 [31.9, 32.5, 30.5, 31.9, 2.31, 72.166],
 [31.8, 32.2, 30.5, 31.4, 2.33, 72.54],
 [31.2, 31.6, 29.8, 30.8, 2.32, 71.114],
 [30.7, 31.1, 29.8, 30.3, 2.14, 65.068],
 [30.4, 33.0, 29.8, 32.4, 2.3, 72.643],
 [32.2, 32.8, 31.1, 32.4, 3.51, 112.143],
 [32.5, 35.2, 32.1, 35.1, 1.99, 66.406],
 [32.2, 33.4, 2

In [23]:
market = "KRW-ASTR"
df = pyupbit.get_ohlcv_from(ticker=market, interval="day", fromDatetime=None,
                   to=None, period=0.5)
df['date'] = df.index.strftime('%Y-%m-%d')
df['market'] = market
df = df[['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']]

values = df.values.tolist()

In [35]:
# KRW 종목 리스트 가져오기
columns = ['date', 'market', 'open', 'high', 'low', 'close', 'volume', 'value']

# 삽입할 테이블 이름
table_name = "c_krw_day"

# 데이터 삽입 쿼리 생성
sql = "INSERT INTO {} ({}) VALUES ({})".format(
    table_name,
    ', '.join(columns),
    ', '.join(['%s'] * len(columns))
)

# 데이터 삽입
mycursor.executemany(sql, values)

# 변경 사항 커밋
mydb.commit()
print(mycursor.rowcount, "레코드가 삽입되었습니다.")
mydb.close()

53 레코드가 삽입되었습니다.


##### 종목마스터 수집

In [ ]:
# SELECT 쿼리 실행
mycursor.execute("SELECT * FROM c_krw_mst")

# 결과 가져오기
result = mycursor.fetchall()

# 튜플의 0번째 인덱스만 가져와서 마켓 리스트 생성
market_list = [t[0] for t in result]

In [38]:
# KRW 종목 리스트 가져오기
# columns = pyupbit.get_tickers(fiat="KRW")

url = "https://api.upbit.com/v1/market/all?isDetails=true"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

mst_json = json.loads(response.text)
df = pd.DataFrame(mst_json)
df = df[['market', 'korean_name', 'english_name', 'market_warning']]
df = df[df['market'].str.contains('KRW')].reset_index(drop=True)


# DB생성용 Master 항목당 최대 길이 출력
max_length = df['market'].str.len().max()
max_string = df[df['market'].str.len() == max_length]['market'].iloc[0]
# print(max_string, len(max_string))

# 마스터에 저장된 종목리스트
mst_market_list = df['market'].tolist()

# DataFrame을 튜플의 리스트로 변환
values = df.values.tolist()

# 컬럼명 리스트
columns = df.columns.tolist()

# 삽입할 테이블 이름
table_name = "c_krw_mst"

# 새로 상장된 종목 유무 확인
new_market = [x for x in mst_market_list if x not in market_list]

In [33]:
# 삽입할 테이블 이름
table_name = "c_krw_mst"

# 데이터 삽입 쿼리 생성
sql = "INSERT INTO {} ({}) VALUES ({})".format(
    table_name,
    ', '.join(columns),
    ', '.join(['%s'] * len(columns))
)

# 데이터 삽입
mycursor.executemany(sql, values)

# 변경 사항 커밋
mydb.commit()
print(mycursor.rowcount, "레코드가 삽입되었습니다.")
mydb.close()

IntegrityError: 1062 (23000): Duplicate entry 'KRW-BTC' for key 'PRIMARY'